<a href="https://colab.research.google.com/github/zamanmiraz/DSandML-Notebooks/blob/main/RAG/07_reranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/guyernest/advanced-rag.git
%cd advanced-rag
!pip install --upgrade -r requirements.txt

Cloning into 'advanced-rag'...
remote: Enumerating objects: 281, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 281 (delta 4), reused 10 (delta 3), pack-reused 268 (from 1)
Receiving objects: 100% (281/281), 18.84 MiB | 11.60 MiB/s, done.
Resolving deltas: 100% (167/167), done.
/content/advanced-rag
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.7/163.7 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
!pip install torchvision==0.18.0
!pip install -q -U google-generativeai

  Using cached torchvision-0.18.0-cp312-cp312-manylinux1_x86_64.whl.metadata (6.6 kB)
  Using cached torch-2.3.0-cp312-cp312-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
Using cached torchvision-0.18.0-cp312-cp312-manylinux1_x86_64.whl (7.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 9.1.0.70
    Uninstalling nvidia-cudnn-cu12-9.1.0.70:
      Successfully uninstalled nvidia-cudnn-cu12-9.1.0.70
  Attempting uninstall: torch
    Found existing installation: torch 2.4.1
    Uninstalling torch-2.4.1:
      Successfully uninstalled torch-2.4.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.23.0+cu126
    Uninstalling torchvision-0.23.0+cu126:
      Successfully u

In [2]:
from rich.console import Console
from rich.style import Style
import pathlib
from rich_theme_manager import Theme, ThemeManager

THEMES = [
    Theme(
        name="dark",
        description="Dark mode theme",
        tags=["dark"],
        styles={
            "repr.own": Style(color="#e87d3e", bold=True),      # Class names
            "repr.tag_name": "dim cyan",                        # Adjust tag names
            "repr.call": "bright_yellow",                       # Function calls and other symbols
            "repr.str": "bright_green",                         # String representation
            "repr.number": "bright_red",                        # Numbers
            "repr.none": "dim white",                           # None
            "repr.attrib_name": Style(color="#e87d3e", bold=True),    # Attribute names
            "repr.attrib_value": "bright_blue",                 # Attribute values
            "default": "bright_white on black"                  # Default text and background
        },
    ),
    Theme(
        name="light",
        description="Light mode theme",
        styles={
            "repr.own": Style(color="#22863a", bold=True),          # Class names
            "repr.tag_name": Style(color="#00bfff", bold=True),     # Adjust tag names
            "repr.call": Style(color="#ffff00", bold=True),         # Function calls and other symbols
            "repr.str": Style(color="#008080", bold=True),          # String representation
            "repr.number": Style(color="#ff6347", bold=True),       # Numbers
            "repr.none": Style(color="#808080", bold=True),         # None
            "repr.attrib_name": Style(color="#ffff00", bold=True),  # Attribute names
            "repr.attrib_value": Style(color="#008080", bold=True), # Attribute values
            "default": Style(color="#000000", bgcolor="#ffffff"),   # Default text and background
        },
    ),
]

theme_dir = pathlib.Path("themes").expanduser()
theme_dir.expanduser().mkdir(parents=True, exist_ok=True)

theme_manager = ThemeManager(theme_dir=theme_dir, themes=THEMES)
theme_manager.list_themes()

dark = theme_manager.get("dark")
theme_manager.preview_theme(dark)
light = theme_manager.get("light")

console = Console(theme=light)

 Theme  Description       Tags  Path               
 dark   Dark mode theme   dark  themes/dark.theme  
 light  Light mode theme        themes/light.theme

                                      Theme: dark - themes/dark.theme                                      
┌───────────────────┬───────────────┬───────┬─────────┬─────────┬────────────────┬────────────────────────┐
│ style             │ color         │ color │ bgcolor │ bgcolor │ attributes     │ example                │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.own          │ #e87d3e       │ █████ │ None    │         │ b------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.tag_name     │ cyan          │ █████ │ None    │         │ -d------------ │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.call         │ bright_yellow │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.str          │ bright_green  │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.number       │ bright_red    │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.none         │ white         │ █████ │ None    │         │ -d------------ │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.attrib_name  │ #e87d3e       │ █████ │ None    │         │ b------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.attrib_value │ bright_blue   │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ default           │ bright_white  │ █████ │ black   │ █████   │ -------------- │ The quick brown fox... │
└───────────────────┴───────────────┴───────┴─────────┴─────────┴────────────────┴────────────────────────┘
┌─ attributes legend ──────────────────────────────────────────────────────────────────┐
│  b: bold, d: dim, i: italic, u: underline, U: double underline, B: blink, 2: blink2  │
│  r: reverse, c: conceal, s: strike, f: frame, e: encircle, o: overline, L: Link      │
└──────────────────────────────────────────────────────────────────────────────────────┘

In [3]:
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

In [4]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
console.print(cross_encoder.model)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (intermediate): BertIntermediate(
            (dense): Linear(in_features=384, out_features=1536, bias=True)
            (intermediate_act_fn): GELUActivation()
          )
          (output): BertOutput(
            (dense): Linear(in_features=1536, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
    )
    (pooler): BertPooler(
      (dense): Linear(in_features=384, out_features=384, bias=True)
      (activation): Tanh()
    )
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (classifier): Linear(in_features=384, out_features=1, bias=True)
)

In [8]:
import json
hybrid_search_results = {}
with open('advanced-rag/data/dense_results.json') as f:
    dense_results = json.load(f)
    for doc in dense_results:
        hybrid_search_results[doc['id']] = doc
with open('advanced-rag/data/sparse_results.json') as f:
    sparse_results = json.load(f)
    for doc in sparse_results:
        hybrid_search_results[doc['id']] = doc
console.print(hybrid_search_results)

{
    15: {
        'id': 15,
        'text': "3.8 â Mixtral_8x7B 3.5 32 > $3.0 i] 228 fos a 2.0 0 5k 10k 15k 20k 25k 30k Context length Passkey 
Performance ry 3.8 â Mixtral_8x7B 3.5 0.8 32 > 0.6 $3.0 i] 228 04 fos 0.2 a 2.0 0.0 OK 4K 8K 12K 16K 20K 24K 28K 0 
5k 10k 15k 20k 25k 30k Seq Len Context length Figure 4: Long range performance of Mixtral. (Left) Mixtral has 100% 
retrieval accuracy of the Passkey task regardless of the location of the passkey and length of the input sequence. 
(Right) The perplexity of Mixtral on the proof-pile dataset decreases monotonically as the context length 
increases.\n\nThe chunk discusses the long-range performance of the Mixtral model, demonstrating its ability to 
retrieve a passkey regardless of its location in a long input sequence, and showing that the model's perplexity on 
the proof-pile dataset decreases as the context length increases.",
        'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}
    },
    4: {
        'id': 4,
        'text': "Instruct under the Apache 2.0 license1, free for academic and commercial usage, ensuring broad 
accessibility and potential for diverse applications. To enable the community to run Mixtral with a fully 
open-source stack, we submitted changes to the vLLM project, which integrates Megablocks CUDA kernels for efficient
inference. Skypilot also allows the deployment of vLLM endpoints on any instance in the cloud. # 2 Architectural 
details Mixtral is based on a transformer architecture [31] and uses the same modifications as described in [18], 
with the notable exceptions that Mix- tral supports a fully dense context length of 32k tokens, and the feed- 
forward blocks are replaced by Mixture-of-Expert layers (Section 2.1). The model architecture parameters are 
summarized in Table 1.\n\nThis chunk describes the architectural details of the Mixtral language model, including 
its use of a transformer architecture with a 32k token context length and mixture-of-expert layers. It also 
mentions the model's open-source licensing and deployment options.",
        'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}
    },
    2: {
        'id': 2,
        'text': 'expertsâ ) to process the token and combine their output additively. This technique increases the 
number of parameters of a model while controlling cost and latency, as the model only uses a fraction of the total 
set of parameters per token. Mixtral is pretrained with multilingual data using a context size of 32k tokens. It 
either matches or exceeds the performance of Llama 2 70B and GPT-3.5, over several benchmarks. In particular, 
Mixture of Experts Layer i gating inputs af outputs router expert\n\nThis chunk describes the key architectural 
details of the Mixtral model, a sparse mixture-of-experts language model that outperforms larger models like Llama 
2 70B and GPT-3.5 on various benchmarks.',
        'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}
    },
    6: {
        'id': 6,
        'text': 'Table 1: Model architecture. # j nâ G(x)i Â· Ei(x). i=0 Here, G(x)i denotes the n-dimensional 
output of the gating network for the i-th expert, and Ei(x) is the output of the i-th expert network. If the gating
vector is sparse, we can avoid computing the outputs of experts whose gates are zero. There are multiple 
alternative ways of implementing G(x) [6, 15, 35], but a simple and performant one is implemented by taking the 
softmax over the Top-K logits of a linear layer [28].\n\nThe chunk describes the architectural details of the 
Mixtral model, specifically the Sparse Mixture of Experts (SMoE) layer that is used in the model.',
        'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}
    },
    7: {
        'id': 7,
        'text': 'We use G(x) := Softmax(TopK(x Â· Wg)), where (TopK(â ))

In [9]:
# This is the query that we used for the retrieval of the above documents
query = "What is context size of Mixtral?"

In [10]:
pairs = [[query, doc['text']] for doc in hybrid_search_results.values()]
scores = cross_encoder.predict(pairs)

console.print(scores)

[  5.065693    3.3688302   7.1048393  -4.116105   -4.375498   -5.261078
  -3.7225776   3.1854541   1.7966672  -2.5144255   2.5638695   2.3361566
  -3.0395977   3.0869865 -11.08604    -8.8575735]

In [11]:
# Combine scores with corresponding document IDs
results_with_scores = [
    (doc_id, hybrid_search_results[doc_id]['text'], score)
    for doc_id, score in zip(hybrid_search_results.keys(), scores)
]

# Sort results by score in descending order and take the top 3
top_results = sorted(results_with_scores, key=lambda x: x[2], reverse=True)[:3]

In [12]:
import numpy as np
from rich.table import Table
table = Table(title="Top 3 Documents after Reranking", show_lines=True)

table.add_column("ID", justify="right", style="cyan", no_wrap=True)
table.add_column("Score", justify="right", style="green", no_wrap=True)
table.add_column("Document", style="#e87d3e")

# Add rows to the table with top 3 results
for doc_id, text, score in top_results:
    table.add_row(str(doc_id), f"{score:.4f}", text)

console.print(table)

                                          Top 3 Documents after Reranking                                          
┏━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ID ┃  Score ┃ Document                                                                                          ┃
┡━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  2 │ 7.1048 │ expertsâ ) to process the token and combine their output additively. This technique increases the │
│    │        │ number of parameters of a model while controlling cost and latency, as the model only uses a      │
│    │        │ fraction of the total set of parameters per token. Mixtral is pretrained with multilingual data   │
│    │        │ using a context size of 32k tokens. It either matches or exceeds the performance of Llama 2 70B   │
│    │        │ and GPT-3.5, over several benchmarks. In particular, Mixture of Experts Layer i gating inputs af  │
│    │        │ outputs router expert                                                                             │
│    │        │                                                                                                   │
│    │        │ This chunk describes the key architectural details of the Mixtral model, a sparse                 │
│    │        │ mixture-of-experts language model that outperforms larger models like Llama 2 70B and GPT-3.5 on  │
│    │        │ various benchmarks.                                                                               │
├────┼────────┼───────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 15 │ 5.0657 │ 3.8 â Mixtral_8x7B 3.5 32 > $3.0 i] 228 fos a 2.0 0 5k 10k 15k 20k 25k 30k Context length Passkey │
│    │        │ Performance ry 3.8 â Mixtral_8x7B 3.5 0.8 32 > 0.6 $3.0 i] 228 04 fos 0.2 a 2.0 0.0 OK 4K 8K 12K  │
│    │        │ 16K 20K 24K 28K 0 5k 10k 15k 20k 25k 30k Seq Len Context length Figure 4: Long range performance  │
│    │        │ of Mixtral. (Left) Mixtral has 100% retrieval accuracy of the Passkey task regardless of the      │
│    │        │ location of the passkey and length of the input sequence. (Right) The perplexity of Mixtral on    │
│    │        │ the proof-pile dataset decreases monotonically as the context length increases.                   │
│    │        │                                                                                                   │
│    │        │ The chunk discusses the long-range performance of the Mixtral model, demonstrating its ability to │
│    │        │ retrieve a passkey regardless of its location in a long input sequence, and showing that the      │
│    │        │ model's perplexity on the proof-pile dataset decreases as the context length increases.           │
├────┼────────┼───────────────────────────────────────────────────────────────────────────────────────────────────┤
│  4 │ 3.3688 │ Instruct under the Apache 2.0 license1, free for academic and commercial usage, ensuring broad    │
│    │        │ accessibility and potential for diverse applications. To enable the community to run Mixtral with │
│    │        │ a fully open-source stack, we submitted changes to the vLLM project, which integrates Megablocks  │
│    │        │ CUDA kernels for efficient inference. Skypilot also allows the deployment of vLLM endpoints on    │
│    │        │ any instance in the cloud. # 2 Architectural details Mixtral is based on a transformer            │
│    │        │ architecture [31] and uses the same modifications as described in [18], with the notable          │
│    │        │ exceptions that Mix- tral supports a fully dense context length of 32k tokens, and the feed-      │
│    │        │ forward blocks are replaced by Mixture-of-Expert layers (Section 2.1). The model architecture     │
│    │        │ parameters are summarized in Table 1.   

In [13]:
# define a variable to hold the search results for the generation model
search_results = [doc[1] for doc in top_results]

In [14]:
# Now time to connect to the large language model
import google.generativeai as genai
from google.colab import userdata
from rich.text import Text

# Configure the Gemini API
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# Initialize the Generative Model
# You can choose a different model if needed
gemini_model = genai.GenerativeModel('gemini-2.5-flash')

# Create the prompt for the Gemini model
prompt = f"""You are chatbot, an research expert. Your top priority is to help guide users to understand research papers.

Based on the following search results, answer the user's query:

Search results:
{search_results}

User query:
{query}
"""

# Generate the response using the Gemini model
response = gemini_model.generate_content(prompt)
response_text = Text(response.text)

# Display the response (optional, you might want to print it directly)
# console.print(response_text)

In [15]:
from rich.panel import Panel

panel = Panel(response_text, title=f"Hybrid Search with Reranking Reply to \"{query}\"")
console.print(panel)

╭─────────────────── Hybrid Search with Reranking Reply to "What is context size of Mixtral?" ────────────────────╮
│ Mixtral has a context size of **32k tokens**.                                                                   │
│                                                                                                                 │
│ It is pretrained with this context size and supports a fully dense context length of 32k tokens.                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯